# LOKI detector test at LARMOR a single run (sample)

Data reduction for single run (sample + direct) to check if the basic procedures work

In [1]:
import scipp as sc
from ess import loki, sans
import scippneutron as scn
import numpy as np

## Define reduction parameters

We define here whether to include the effects of gravity, and the binning in wavelength and in $Q$ to be used.

In [2]:
# Include effects of gravity?
gravity = True


wavelength_bins = sc.linspace(dim='wavelength', start=0.9, stop=13.5, num=110,
                              unit='angstrom')

q_bins = sc.linspace(dim='Q', start=0.008, stop=0.6, num=55, unit='1/angstrom')

# Define the range of wavelengths for the monitors that are considered
# to not be part of the background
monitor_non_background_range = sc.array(dims=['wavelength'],
                                        values=[0.7, 17.1], unit='angstrom')

## Loading data files

We load a sample measurement file and a direct measurement file (currently same file).
For both files, only the first quarter of pixels will be used, as the rest are used for monitoring.

In [3]:
input_file = 'Larmor_data/60322-2022-03-02_2205.nxs'
fixed_file = f'{input_file[:-4]}_fixed.nxs'

path='Larmor_data'
# Sample measurement
sample =  scn.load_nexus(data_file=f'{path}/{fixed_file}')
# Direct measurement is with the empty sample holder/cuvette
direct = scn.load_nexus(data_file=f'{path}/{fixed_file}')
#Adding sample position not written to the file
sample

/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess/lib/python3.8/site-packages/scippneutron/file_loading/_monitor_data.py:31: UserWarning: Event data present in NXMonitor group /entry/instrument/monitor_1. Histogram-mode monitor data from this group will be ignored.
  warnings.warn(f"Event data present in NXMonitor group {group.name}. "
/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess/lib/python3.8/site-packages/scippneutron/file_loading/_monitor_data.py:31: UserWarning: Event data present in NXMonitor group /entry/instrument/monitor_2. Histogram-mode monitor data from this group will be ignored.
  warnings.warn(f"Event data present in NXMonitor group {group.name}. "


<scipp.DataArray>
Dimensions: Sizes[detector_id:401408, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [1, 2, ..., 401407, 401408]
  position                  vector3              [m]  (detector_id)  [(0.499023, 0.41833, -4.42986), (0.500969, 0.41833, -4.42986), ..., (0.993148, 1.97683, -4.35336), (0.995093, 1.97683, -4.35336)]
  source_position           vector3              [m]  ()  [(0, 0, 25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [34, 100004725]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Attributes:
  end_time                   string  [dimensionless]  ()  ["2022-02-23T10:10:00.000Z"]
  experiment_title           string  [dimensionless]  ()  ["My experiment"]
  instrument_name            string  [dimensionless]  ()  ["LARMOR"]
  monitor_1               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [1]
  tof                         int32             [ns]  (tof [bin-edge])  [3418, 99958163]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:12020892, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-02-23T09:59:59.886470753, 2022-02-23T09:59:59.886470753, ..., 2022-02-23T10:10:00.014872502, 2022-02-23T10:10:00.014872502]
  tof                         int32             [ns]  (event)  [99808149, 99858153, ..., 44305522, 44355526]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  monitor_2               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [1]
  tof                         int32             [ns]  (tof [bin-edge])  [3418, 99958163]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:12020892, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-02-23T09:59:59.886470753, 2022-02-23T09:59:59.886470753, ..., 2022-02-23T10:10:00.014872502, 2022-02-23T10:10:00.014872502]
  tof                         int32             [ns]  (event)  [99808149, 99858153, ..., 44305522, 44355526]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  start_time                 string  [dimensionless]  ()  ["2022-02-23T10:00:00.000Z"]

### Adding sample position and converting to microseconds

In [4]:
sample.coords['sample_position'] = sc.vector(value = np.array([0, 0, 9.0]), unit = sc.units.m)
direct.coords['sample_position'] = sc.vector(value = np.array([0, 0, 9.0]), unit = sc.units.m)
sample.bins.coords['tof'] = sample.bins.coords['tof'].to(unit='us', dtype='float64')
direct.bins.coords['tof'] = direct.bins.coords['tof'].to(unit='us', dtype='float64')
sample.coords['tof'] = sample.coords['tof'].to(unit='us', dtype='float64')
direct.coords['tof'] = direct.coords['tof'].to(unit='us', dtype='float64')

## Extract monitors

From these two runs, we extract the data from the incident and transmission monitors,
and place them in their own `dict`, as this will be useful further down.

In [5]:
monitors = {
    'sample': {'incident': sample.attrs["monitor_1"].value,
               'transmission': sample.attrs["monitor_2"].value},
    'direct': {'incident': direct.attrs["monitor_1"].value,
               'transmission': direct.attrs["monitor_2"].value}
}

In [ ]:
result = sc.histogram(monitors, wavelength_bins)

### Adding monitor position manually. To be resolved between ECDC and scipp

In [6]:
monitors['sample']['incident'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['sample']['transmission'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)

monitors['direct']['incident'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['direct']['transmission'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)

monitors['sample']['incident'].coords['source_position'] = sc.vector(value = np.array([0, 0, 25.3]), unit = sc.units.m)
monitors['sample']['transmission'].coords['source_position'] = sc.vector(value = np.array([0, 0, 25.3]), unit = sc.units.m)

monitors['direct']['incident'].coords['source_position'] = sc.vector(value = np.array([0, 0, 25.3]), unit = sc.units.m)
monitors['direct']['transmission'].coords['source_position'] = sc.vector(value = np.array([0, 0, 25.3]), unit = sc.units.m)

## Apply offsets to pixel positions

**Note:** for production ESS Nexus files are produced, this step should go away.

**Note:** The corrections also include adjusting the beam center position,
which in the future will be determined using helpers in the `ess.sans` package.

Various positions for the sample holder, detector pixels, and monitors are incorrect in the Nexus files.
The step below corrects this.
We also add the shape of the pixels, which is missing from the geometry information,
and is required to compute the solid angle for each detector pixel.

In [7]:
# Custom position offsets

sample_pos_z_offset = 0.3053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m

# Geometry transformation based on the found beam center position 
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m

In [8]:
# Add pixel shapes
sample.coords["pixel_width"] = 0.0075 * sc.units.m
sample.coords["pixel_height"] = 0.0117188 * sc.units.m

# Change sample position
sample.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
sample.coords["position"].fields.y += bench_pos_y_offset
# Now shift pixels positions to get the correct beam center
sample.coords['position'].fields.x += x_offset
sample.coords['position'].fields.y += y_offset

## Masking

The next step is to mask noisy and saturated pixels,
as well as a time-of-flight range that contains spurious artifacts from the beamline components.

**Note:** We use programatic masks here and not those stored in xml files.

### Mask bad pixels

We mask the edges of the square-shaped detector panel with a simple distance relation.
We also mask the region close to the beam center, where pixels are saturated.

In [ ]:
x = sample.coords['position'].fields.x
y = sample.coords['position'].fields.y
mask_center = sc.less(sc.sqrt(x*x+y*y), 0.045 * sc.units.m)
mask_edges = sc.greater(sc.abs(x), 0.36 * sc.units.m) # roughly all det IDs listed in original

sample.masks['edges'] = mask_edges
sample.masks['center'] = mask_center

A good sanity check is to view the masks on the instrument view:

In [ ]:
#scn.instrument_view(sample, pixel_size=0.0075)

### Mask in time-of-flight

We will now take out the time regions with Bragg peaks from the beam stop and detector window,
although in reality the peaks appear only close to the beam stop,
and will make little difference to $I(Q)$.

This could be implemented as masking specific time bins for a specific region in space,
but for now we keep it simple.

In [9]:
sample

<scipp.DataArray>
Dimensions: Sizes[detector_id:401408, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1, 2, ..., 401407, 401408]
  pixel_height              float64              [m]  ()  [0.0117188]
  pixel_width               float64              [m]  ()  [0.0075]
  position                  vector3              [m]  (detector_id)  [(-0.591903, -0.35072, 4.43214), (-0.589958, -0.35072, 4.43214), ..., (-0.0977792, 1.20778, 4.50864), (-0.0958339, 1.20778, 4.50864)]
  sample_position           vector3              [m]  ()  [(0, 0, 9.3053)]
  source_position           vector3              [m]  ()  [(0, 0, -25.3)]
  tof                       float64            [µs]  (tof [bin-edge])  [0.034, 100005]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Attributes:
  base_position             vector3              [m]  (detector_id)  [(0, -0.00767008, 0.00113946), (0.00194537, -0.00767008, 0.00113946), ..., (0.494124, 1.55083, 0.0776375), (0.49607, 1.55083, 0.0776375)]
  end_time                   string           [None]  ()  ["2022-02-23T10:10:00.000Z"]
  experiment_title           string           [None]  ()  ["My experiment"]
  instrument_name            string           [None]  ()  ["LARMOR"]
  monitor_1               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1]
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [3418, 99958163]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:12020892, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-02-23T09:59:59.886470753, 2022-02-23T09:59:59.886470753, ..., 2022-02-23T10:10:00.014872502, 2022-02-23T10:10:00.014872502]
  tof                         int32             [ns]  (event)  [99808149, 99858153, ..., 44305522, 44355526]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  monitor_2               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1]
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [3418, 99958163]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:12020892, ]
Coordinates:
  pulse_time              datet

In [10]:
#Less than 1500 may still neeed to be fixed.
#tof = data.coords['tof']
#tof_masked_region = sc.less(tof['tof',1:], 1500.0 * sc.units.us) | \
#                         (sc.greater(tof['tof',:-1], 17500.0 * sc.units.us) & \
#                          sc.less(tof['tof',1:], 19000.0 * sc.units.us))



mask_tof_min = sc.scalar(17500.0, unit='us')
mask_tof_max = sc.scalar(19000.0, unit='us')
tof_masked_region = sc.concat([sample.coords['tof']['tof', 0],
                               mask_tof_min, mask_tof_max,
                               sample.coords['tof']['tof', -1]], dim='tof')

sample = sc.bin(sample, edges=[tof_masked_region])
sample.masks['bragg_peaks'] = sc.array(dims=['tof'], values=[False, True, False])
sample

<scipp.DataArray>
Dimensions: Sizes[detector_id:401408, tof:3, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1, 2, ..., 401407, 401408]
  pixel_height              float64              [m]  ()  [0.0117188]
  pixel_width               float64              [m]  ()  [0.0075]
  position                  vector3              [m]  (detector_id)  [(-0.591903, -0.35072, 4.43214), (-0.589958, -0.35072, 4.43214), ..., (-0.0977792, 1.20778, 4.50864), (-0.0958339, 1.20778, 4.50864)]
  sample_position           vector3              [m]  ()  [(0, 0, 9.3053)]
  source_position           vector3              [m]  ()  [(0, 0, -25.3)]
  tof                       float64            [µs]  (tof [bin-edge])  [0.034, 17500, 19000, 100005]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                       float64            [µs]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Masks:
  bragg_peaks                  bool           [None]  (tof)  [False, True, False]
Attributes:
  base_position             vector3              [m]  (detector_id)  [(0, -0.00767008, 0.00113946), (0.00194537, -0.00767008, 0.00113946), ..., (0.494124, 1.55083, 0.0776375), (0.49607, 1.55083, 0.0776375)]
  end_time                   string           [None]  ()  ["2022-02-23T10:10:00.000Z"]
  experiment_title           string           [None]  ()  ["My experiment"]
  instrument_name            string           [None]  ()  ["LARMOR"]
  monitor_1               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1]
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [3418, 99958163]
Data:
                          DataArrayView           [None]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:12020892, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2022-02-23T09:59:59.886470753, 2022-02-23T09:59:59.886470753, ..., 2022-02-23T10:10:00.014872502, 2022-02-23T10:10:00.014872502]
  tof                         int32             [ns]  (event)  [99808149, 99858153, ..., 44305522, 44355526]
Data:
                            float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]

]

]
  monitor_2               DataArray           [None]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:1, tof:1, ]
Coordinates:
  detector_id                 int32           [None]  (detector_id)  [1]
  position                  vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [3418, 99958163]
Data:
                          DataArrayView           [None]  (detector_id, to

In [ ]:
sc.plot(sample)

## Coordinate transformation graph

To compute the wavelength $\lambda$, the scattering angle $\theta$, and the $Q$ vector for our data,
we construct a coordinate transformation graph.

It is based on classical conversions from `tof` and pixel `position` to $\lambda$ (`wavelength`),
$\theta$ (`theta`) and $Q$ (`Q`),
but takes into account the Earth's gravitational field, which bends the flight path of the neutrons,
to compute the scattering angle $\theta$.

The angle can be found using the following expression ([Seeger & Hjelm 1991](#seeger1991))

$$\theta = \frac{1}{2}\sin^{-1}\left(\frac{\sqrt{ x^{2} + \left( y + \frac{g m_{\rm n}}{2 h^{2}} \lambda^{2} L_{2}^{2} \right)^{2} } }{L_{2}}\right)$$

where $x$ and $y$ are the spatial coordinates of the pixels in the horizontal and vertical directions, respectively,
$m_{\rm n}$ is the neutron mass,
$L_{2}$ is the distance between the sample and a detector pixel,
$g$ is the acceleration due to gravity,
and $h$ is Planck's constant.

The monitors require a slightly different conversion graph,
as the flight path of the neutrons hitting them does not scatter through the sample,
it links the source to the monitor with a straight line.

The conversion graphs for the detectors and the monitors are defined in the `sans` module,
and can be obtained via

In [13]:
data_graph, monitor_graph = sans.i_of_q.make_coordinate_transform_graphs(gravity=gravity)
#sc.show_graph(data_graph, simplified=True)

## Convert the data to wavelength

To compute the wavelength of the neutrons,
we use Scipp's `transform_coords` method by supplying our graph defined above
(see [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
for more information about using `transform_coords`) to a helper conversion function.

In [14]:
sample, monitors = sans.i_of_q.convert_to_wavelength(
    data=sample,
    monitors=monitors,
    data_graph=data_graph,
    monitor_graph=monitor_graph)

## Compute normalization term

In a SANS experiment, the scattering cross section $I(Q)$ is defined as ([Heenan et al. 1997](#heenan1997))

$$ I(Q) = \frac{\partial\Sigma{Q}}{\partial\Omega} = \frac{A_{H} \Sigma_{R,\lambda\subset Q} C(R, \lambda)}{A_{M} t \Sigma_{R,\lambda\subset Q}M(\lambda)T(\lambda)D(\lambda)\Omega(R)} $$

where $A_{H}$ is the area of a mask (which avoids saturating the detector) placed between the monitor of area $A_{M}$ and the main detector.
$\Omega$ is the detector solid angle, and $C$ is the count rate on the main detector, which depends on the position $R$ and the wavelength.
$t$ is the sample thickness, $M$ represents the incident monitor count rate, and $T$ is known as the transmission fraction.
Finally, $D$ is the 'direct beam function', and is defined as

$$ D(\lambda) = \frac{\eta(\lambda)}{\eta_{M}(\lambda)} \frac{A_{H}}{A_{M}} $$

where $\eta$ and $\eta_{M}$ are the detector and monitor efficiencies, respectively.

Hence, in order to normalize the main detector counts $C$, we need compute the transmission fraction $T(\lambda)$,
the direct beam function $D(\lambda)$ and the solid angle $\Omega(R)$.

### Transmission fraction

The transmission fraction is computed from the monitor counts.
It essentially compares the neutron counts before the sample, and after the sample,
to give an absorption profile of the sample as a function of wavelength.

It is defined as the ratio of counts between on the transmission monitor to the counts on the incident monitor for the sample run,
multiplied by the inverse ratio for the direct run, i.e.

$$ T(\lambda) = \frac{M_{\rm sample}^{\rm transmission}}{M_{\rm sample}^{\rm incident}} \frac{M_{\rm direct}^{\rm incident}}{M_{\rm direct}^{\rm transmission}} $$

#### Remove background noise and rebin monitors

To compute the ratios, the monitor counts are first cleaned of background noise counts.
By looking at the monitors on a plot,

In [15]:
p = sc.plot(monitors['sample'], norm='log')
#sc.plot(monitors['direct'], norm='log', ax=p.ax, color=['C2', 'C3'])
p

/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess_0.4/lib/python3.8/site-packages/scipp/plotting/figure2d.py:147: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  self.image_colors.set_extent(new_values["extent"])
/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess_0.4/lib/python3.8/site-packages/scipp/plotting/view2d.py:88: RuntimeWarning: invalid value encountered in true_divide
  diffy = np.abs(self.current_lims[dimy] - xylims[dimy]) / dy
/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess_0.4/lib/python3.8/site-packages/scipp/plotting/figure2d.py:148: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  self.image_values.set_extent(new_values["extent"])
/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess_0.4/lib/python3.8/site-packages/scipp/plotting/view2d.py:88: RuntimeWarning: invalid value encountered in true_divide
  diffy = np.abs

we define a valid wavelength range between 0.7 &#8491; and 17.1 &#8491;.

In [16]:
non_background_range = sc.array(dims=['wavelength'], values=[0.7, 17.1], unit='angstrom')

In [18]:
monitors

{'sample': {'incident': <scipp.DataArray>
  Dimensions: Sizes[detector_id:1, wavelength:1, ]
  Coordinates:
    detector_id                 int32           [None]  (detector_id)  [1]
    wavelength                float64             [Å]  (wavelength [bin-edge])  [0.000534456, 15.63]
  Data:
                            DataArrayView           [None]  (detector_id, wavelength)  [<scipp.DataArray>
  Dimensions: Sizes[event:12020892, ]
  Coordinates:
    pulse_time              datetime64             [ns]  (event)  [2022-02-23T09:59:59.886470753, 2022-02-23T09:59:59.886470753, ..., 2022-02-23T10:10:00.014872502, 2022-02-23T10:10:00.014872502]
    wavelength                float64             [Å]  (event)  [15.6065, 15.6143, ..., 6.92783, 6.93565]
  Data:
                              float32         [counts]  (event)  [1, 1, ..., 1, 1]  [1, 1, ..., 1, 1]
  Attributes:
    tof                         int32             [ns]  (event)  [99808149, 99858153, ..., 44305522, 44355526]
  
  ]
  Att

and we remove the mean background counts and rebin the monitors in a single step using

In [17]:
#TODO: Here we will need to change for evenet data replace bininning with histograming (if monitor_1.bins == None)
monitors = sans.i_of_q.denoise_and_rebin_monitors(
    monitors=monitors,
    wavelength_bins=wavelength_bins,
    non_background_range=non_background_range)

BinnedDataError: Binary operations such as '+' with binned data are only supported with dtype=VariableView, got dtype=DataArrayView. See https://scipp.github.io/user-guide/binned-data/computation.html#Event-centric-arithmetic for equivalent operations for binned (event) data.

The transmission fraction is then computed by using 

In [ ]:
transmission_fraction = sans.normalization.transmission_fraction(
    data_monitors=monitors['sample'], direct_monitors=monitors['direct'])
transmission_fraction

In [ ]:
transmission_fraction.plot()

### Direct beam function

The direct beam function is a parameter of the instrument that is well-known to the instrument scientist,
and does not vary much over time.
It is usually stored in a file, and updated a small number of times per year.

Here, we load the direct beam function for the SANS2D instrument from file,
and perform an interpolation so that it spans the same wavelength range as the one requested at the top of the notebook.

In [ ]:
# Load direct beam function for main detector
direct_beam = loki.io.load_rkh_wav(loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))

direct_beam = sans.i_of_q.resample_direct_beam(
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins)

sc.plot(direct_beam)

### Solid Angle

The `sans.normalization` module also provides a utility to compute the solid angles of rectangular detector pixels:

In [ ]:
solid_angle = sans.normalization.solid_angle_of_rectangular_pixels(
    sample,
    pixel_width=sample.coords['pixel_width'],
    pixel_height=sample.coords['pixel_height'])
solid_angle

### The denominator term

We combine all the terms above to compute the `denominator`.
We then attach to the denominator some coordinates required to perform the conversion to $Q$.

In [ ]:
denominator = sans.normalization.compute_denominator(
        direct_beam=direct_beam,
        data_incident_monitor=monitors['sample']['incident'],
        transmission_fraction=transmission_fraction,
        solid_angle=solid_angle)
# Insert a copy of coords needed for conversion to Q.
# TODO: can this be avoided by copying the Q coords from the converted numerator?
for coord in ['position', 'sample_position', 'source_position']:
    denominator.coords[coord] = sample.meta[coord]

denominator

In [ ]:
sc.plot(denominator.sum('spectrum'), norm='log')

## Convert to Q

Using the coordinate transformation graph as above,
we can compute the momentum vector $Q$, and then merge all the events in the detector pixel bins,
so as to obtain an intensity that depends only on $Q$.

This is done with the `convert_to_q_and_merge_spectra` helper.

In [ ]:
wavelength_bands = sc.concat(
            [wavelength_bins.min(), wavelength_bins.max()], dim='wavelength')

In [ ]:
sample_q = sans.i_of_q.convert_to_q_and_merge_spectra(
    data=sample,
    graph=data_graph,
    wavelength_bands=wavelength_bands,
    q_bins=q_bins,
    gravity=gravity)

In [ ]:
sc.plot(sample_q, norm='log')

### Convert denominator to Q

Converting the denominator to $Q$ is achieved in the same way

In [ ]:
denominator_q = sans.i_of_q.convert_to_q_and_merge_spectra(
    data=denominator,
    graph=data_graph,
    wavelength_bands=wavelength_bands,
    q_bins=q_bins,
    gravity=True)

sc.plot(denominator_q, norm='log')

## Normalize the sample

Finally, we normalize the sample with the denominator as a function of $Q$.

In [ ]:
sample_normalized = sans.normalization.normalize(
    numerator=sample_q,
    denominator=denominator_q)
sample_normalized

In [ ]:
sc.plot(sample_normalized)

## References

<div id='heenan1997'></div>

Heenan R. K., Penfold J., King S. M., **1997**,
*SANS at Pulsed Neutron Sources: Present and Future Prospects*,
[J. Appl. Cryst., 30, 1140-1147](https://doi.org/10.1107/S0021889897002173)

<div id='manasi2021'></div>

Manasi I., Andalibi M. R., Atri R. S., Hooton J., King S. M., Edler K. J., **2021**,
*Self-assembly of ionic and non-ionic surfactants in type IV cerium nitrate and urea based deep eutectic solvent*,
[J. Chem. Phys. 155, 084902](https://doi.org/10.1063/5.0059238)

<div id='seeger1991'></div>

Seeger P. A., Hjelm R. P. Jnr, **1991**,
*Small-angle neutron scattering at pulsed spallation sources*,
[J. Appl. Cryst., 24, 467-478](https://doi.org/10.1107/S0021889891004764)